# Convex Hull Vs Minimum Bounding Box
The basic idea is given a set of points, find the wrapping points and reconstruct a 3D mesh of the shape

### Imports

In [1]:
from __future__ import division
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import sys  # maxint
from mpl_toolkits.mplot3d import Axes3D
from math import *
%matplotlib inline
import matplotlib as mpl

In [2]:
%matplotlib qt

### Bounding Boxes

In [3]:
def minBoundingRect(hull_points_3d,Epsilon):
    a=[]
    boxes=[]
    count = 0
    # Test each angle to find bounding box with smallest area
    min_bbox = (0, 0, sys.maxsize, 0, 0, 0, 0, 0, 0, 0, 0, 0) 
    # rot_angle1, rot_angle2, volume, width, height, depth, min_x, max_x, min_y, max_y, max_z, min_z

    angi=np.arange(0.0, 90, Epsilon)

    for iDeg in (angi):
        boxes.append([])
        angj=np.arange(0.0, 180, Epsilon)
        for jDeg in (angj):
            
            i=deg2rad(iDeg)
            j=deg2rad(jDeg)
            
            #Rotation Matrix = Rx * Rz
            R = array([ [ math.cos(i),                                       -math.cos(i-(math.pi/2)),              0],
                        [ math.cos(j)*math.cos(i-(math.pi/2))  ,             math.cos(j)*math.cos(i), -math.cos(j-(math.pi/2)) ],
                        [ math.cos(i-(math.pi/2)) * math.cos(j-(math.pi/2)), math.cos(j-(math.pi/2))*math.cos(i),  math.cos(j) ] ])

            #             OR
            #             Rx = array([ [ 1,0,0],
            #                          [ 0, math.cos(j), -math.cos(j-(math.pi/2)) ], 
            #                          [ 0, math.cos(j-(math.pi/2)), math.cos(j) ] ])
            #             Rz = array([ [ math.cos(i), -math.cos(i-(math.pi/2)), 0 ], 
            #                          [ math.cos(i-(math.pi/2)), math.cos(i) , 0 ], 
            #                          [ 0,0,1]])
            #             R = matmul(Rx,Rz)
                        
            # Apply this rotation to convex hull points
            rot_points = matmul((hull_points_3d), R )

            a.append(rot_points)

            # Find min/max x,y points
            min_x = nanmin(rot_points[:,0], axis=0)
            max_x = nanmax(rot_points[:,0], axis=0)
            min_y = nanmin(rot_points[:,1], axis=0)
            max_y = nanmax(rot_points[:,1], axis=0)
            min_z = nanmin(rot_points[:,2], axis=0)
            max_z = nanmax(rot_points[:,2], axis=0)

            # Calculate height/width/area of this bounding rectangle
            width = max_x - min_x
            height = max_y - min_y
            depth = max_z-min_z
            Volume = width * height * depth

            boxes[count].append(Volume)

            # Store the smallest rect found first (a simple convex hull might have 2 answers with same area)
            if (Volume < min_bbox[2]):
                min_bbox = ( i,j, Volume, width, height, depth, min_x, max_x, min_y, max_y, min_z, max_z )
        count=count+1


        # Re-create rotation matrix for smallest rect
    i = min_bbox[0]
    j = min_bbox[1]
    R = array([ [ math.cos(i),                                       -math.cos(i-(math.pi/2)),              0],
                [ math.cos(j)*math.cos(i-(math.pi/2))  ,             math.cos(j)*math.cos(i), -math.cos(j-(math.pi/2)) ],
                [ math.cos(i-(math.pi/2)) * math.cos(j-(math.pi/2)), math.cos(j-(math.pi/2))*math.cos(i),  math.cos(j)             ] ])

    # Project convex hull points onto rotated frame
    rot_points = matmul((hull_points_3d), R )

    # min/max x,y points are against baseline
    min_x = min_bbox[6]
    max_x = min_bbox[7]
    min_y = min_bbox[8]
    max_y = min_bbox[9]
    min_z = min_bbox[10]
    max_z = min_bbox[11]

    # Calculate center point and project onto rotated frame
    center_x = (min_x + max_x)/2
    center_y = (min_y + max_y)/2
    center_z = (min_x + max_x)/2
    center_point = matmul(linalg.inv(R) , [ center_x, center_y, center_z])
    #print ("Bounding box center point: \n", center_point)

    # Calculate corner points and project onto rotated frame
    corner_points = zeros( (16,3) ) # empty 2 column array
    corner_points[0] = matmul(  (R) ,[ min_x, min_y ,min_z ])
    corner_points[1] = matmul(  (R), [ max_x, min_y ,min_z ])
    corner_points[2] = matmul(  (R), [ max_x, max_y ,min_z ])
    corner_points[3] = matmul(  (R), [ min_x, max_y ,min_z ])
    corner_points[4] = matmul(  (R), [ min_x, min_y ,min_z ])
    
    corner_points[5] = matmul( (R), [ min_x, min_y ,max_z ])
    corner_points[6] = matmul( (R), [ max_x, min_y ,max_z ])
    corner_points[7] = matmul( (R), [ max_x, max_y ,max_z ])
    corner_points[8] = matmul( (R), [ min_x, max_y ,max_z ])
    corner_points[9] = matmul( (R), [ min_x, min_y ,max_z ])

    corner_points[10] = matmul( (R), [ max_x, min_y ,max_z ])
    corner_points[11] = matmul( (R), [ max_x, min_y ,min_z ])
    
    corner_points[12] = matmul( (R), [ max_x, max_y ,min_z ] )
    corner_points[13] = matmul( (R), [ max_x, max_y ,max_z ])

    corner_points[14] = matmul( (R), [ min_x, max_y ,max_z ])
    corner_points[15] = matmul( (R), [ min_x, max_y ,min_z ])

    return ( i, j, min_bbox[2], min_bbox[3], min_bbox[4], min_bbox[5], center_point, corner_points,a,boxes,rot_points) 
    # rot_angle, area, width, height, center_point, corner_points

## Generate a random(ish) set of points and Variables Initialization

In [39]:
n = 10 #Number of Input points
Scale = 100 # Scale of Given Points

# Cube points
# sphere_points = array([ [0,0,0], [0,1,0], [1,1,0], [1,0,0], [1,0,1], [0,0,1], [0,1,1], [1,1,1] ])
# sphere_points = Scale * sphere_points

# Simplex points
# sphere_points = array([[0,0,0], [0,1,-1], [1,0,-1], [1,1,0] ])
sphere_points = array([ [0,0,1], [0,1,0], [1,0,0], [1,1,1] ])
sphere_points = Scale * sphere_points

# Dual Cube points
# sphere_points = array([ [1,0,0], [0,1,0], [0,0,1], [-1,0,0], [0,-1,0], [0,0,-1] ])
# sphere_points = Scale * sphere_points

# Random points

# sphere_points = Scale*random.exponential(1,size = (n,3) )
# sphere_points = Scale*random.random(size = (n,3) )

fig = plt.figure(1)
ax = fig.subplots(1, 1, subplot_kw={'projection':'3d'})
ax.scatter(sphere_points[:, 0], sphere_points[:, 1], sphere_points[:, 2], s=20, c='b', zorder=10)
ax.set_title('Given Set of Points')

Text(0.5, 0.92, 'Given Set of Points')

## Bounding Box Calculation

In [40]:
Epsilon = 0.5 #Angle Variations for bounding boxes
(rot_angle1, rot_angle2, Volume, width, height, depth, center_point, corner_points,a,boxes,
 rot_points) = minBoundingRect(sphere_points,Epsilon)

print ("Minimum Volume bounding box:")
print ("Rotation angles:")
print("Along z axes: ", rot_angle1, "rad  (", rot_angle1*(180/math.pi), "deg )")
print("Along x axes: ", rot_angle2, "rad  (", rot_angle2*(180/math.pi), "deg )")
print ("Width:", width, " Height:", height, " Depth:", depth, "  Volume:", Volume)
print ("Center point: \n", center_point) # numpy array
print ("Corner points: \n", corner_points, "\n")  # numpy array

fig = plt.figure(4)
ax = fig.add_subplot(111, projection='3d')
ax.plot3D(corner_points[:, 0], corner_points[:, 1], corner_points[:, 2], 'gray')
ax.scatter(sphere_points[:, 0], sphere_points[:, 1], sphere_points[:, 2], s=20, c='b', zorder=10)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('Minimum Volume Bounding Box')

Minimum Volume bounding box:
Rotation angles:
Along z axes:  0.0 rad  ( 0.0 deg )
Along x axes:  0.0 rad  ( 0.0 deg )
Width: 100.0  Height: 100.0  Depth: 100.0   Volume: 1000000.0
Center point: 
 [50. 50. 50.]
Corner points: 
 [[ 7.49879891e-31 -6.12323400e-15 -6.12323400e-15]
 [ 1.00000000e+02  3.74939946e-31 -6.12323400e-15]
 [ 1.00000000e+02  1.00000000e+02  0.00000000e+00]
 [-6.12323400e-15  1.00000000e+02  0.00000000e+00]
 [ 7.49879891e-31 -6.12323400e-15 -6.12323400e-15]
 [ 7.49879891e-31 -1.22464680e-14  1.00000000e+02]
 [ 1.00000000e+02 -6.12323400e-15  1.00000000e+02]
 [ 1.00000000e+02  1.00000000e+02  1.00000000e+02]
 [-6.12323400e-15  1.00000000e+02  1.00000000e+02]
 [ 7.49879891e-31 -1.22464680e-14  1.00000000e+02]
 [ 1.00000000e+02 -6.12323400e-15  1.00000000e+02]
 [ 1.00000000e+02  3.74939946e-31 -6.12323400e-15]
 [ 1.00000000e+02  1.00000000e+02  0.00000000e+00]
 [ 1.00000000e+02  1.00000000e+02  1.00000000e+02]
 [-6.12323400e-15  1.00000000e+02  1.00000000e+02]
 [-6.123

Text(0.5, 0.92, 'Minimum Volume Bounding Box')

## Calculate the Convex Hull and draw the points. SCIPY REQUIRED

In [41]:
from scipy.spatial import ConvexHull

# Calculate the convex hull (i.e wrapping points) for the set of points
hull = ConvexHull(sphere_points)

print("Convex hull vertices: ",sphere_points[hull.vertices].shape[0])

# Plot it (red is the convex hull points, i.e the wrapping points and grey is the rest)
fig = plt.figure(2)
ax = fig.subplots(1, 1, subplot_kw={'projection':'3d'})

# Plot all the points
ax.scatter(sphere_points[:, 0], sphere_points[:, 1], sphere_points[:, 2], s=2, c='grey', zorder=10)

# Plot the convex hull points in red
ax.scatter(sphere_points[hull.vertices, 0], 
           sphere_points[hull.vertices, 1], 
           sphere_points[hull.vertices, 2], 
           s=20, c='r', zorder=20)
ax.set_title('Convex Hull')

Convex hull vertices:  4


Text(0.5, 0.92, 'Convex Hull')

## Use the results from the Convex Hull to draw a triangular surface

In [42]:
fig = plt.figure(3)
ax = fig.subplots(1, 1, subplot_kw={'projection':'3d'})
ax.plot_trisurf(sphere_points[:, 0], 
                sphere_points[:, 1], 
                sphere_points[:, 2], 
                triangles=hull.simplices, 
                cmap=plt.cm.Spectral)
ax.plot3D(corner_points[:, 0], corner_points[:, 1], corner_points[:, 2], 'gray')
ax.scatter(sphere_points[:, 0], sphere_points[:, 1], sphere_points[:, 2], s=20, c='b', zorder=10)
ax.set_title('Convex Hull')
volume = ConvexHull(sphere_points).volume
print("Convex Hull Volume: ", volume)
print("Ratio: ", Volume/volume)


Convex Hull Volume:  333333.3333333333
Ratio:  3.0


## Graph for Volume Relation with Angles

In [43]:
Z=boxes
# %matplotlib qt
x = np.linspace(0, 90, np.asarray(Z).shape[1])
y = np.linspace(0, 180, np.asarray(Z).shape[0])

X, Y = np.meshgrid(x, y)
fig = plt.figure(5)
ax = plt.axes(projection='3d')
surf = ax.plot_surface(X, Y, np.asarray(Z), cmap=mpl.cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_xlabel('Angle along z axis')
ax.set_ylabel('Angle along x axis')
ax.set_zlabel('Volume')
ax.set_title('Volume as a function of Angles (z and x)')

Text(0.5, 0.92, 'Volume as a function of Angles (z and x)')

## Save Volume Matrix

In [13]:
mat = np.matrix(Z)
with open('outfile.txt','wb') as f:
    for line in mat:
        np.savetxt(f, line, fmt='%.2f')